In [1]:
import yaml
import wandb

config_path = "configs/base_config.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

wandb.init(project=config['wandb']['project'],
           entity=config['wandb']['entity'],
           config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: balamanikandan (tangoeye). Use `wandb login --relogin` to force relogin


In [2]:
import torchreid
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [3]:
model = torchreid.models.build_model(
    name        = config['model']['name'],
    num_classes = 2000 if 'tango2000' in config['data']['sources'] else datamanager.num_train_pids, #2000 for Tango
    loss        = config['loss']['name'],
    pretrained  = config['model']['pretrained'],
    pooling     = config['model']['final_pooling'],
    **config['custom']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim=config['train']['optim'],
    lr=config['train']['lr']
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    **config['train']['lr_scheduler'],
)

gem


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [6]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=config['loss']['softmax']['label_smooth'],
    logger=wandb
)

In [7]:
engine.run(
    save_dir=f"log/{config['model']['name']}_{config['data']['sources'][0]}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.175 (0.189)	data 0.000 (0.005)	eta 0:18:58	loss_t 0.6585 (0.4862)	loss_x 5.9408 (6.5798)	acc 4.6875 (2.7188)	lr 0.000300
epoch: [1/60][100/101]	time 0.176 (0.183)	data 0.000 (0.003)	eta 0:18:10	loss_t 0.2831 (0.4286)	loss_x 4.5800 (5.9520)	acc 21.0938 (8.0938)	lr 0.000300
epoch: [2/60][50/101]	time 0.171 (0.175)	data 0.000 (0.004)	eta 0:17:15	loss_t 0.4691 (0.3240)	loss_x 3.4451 (3.9812)	acc 42.9688 (33.4531)	lr 0.000300
epoch: [2/60][100/101]	time 0.179 (0.174)	data 0.000 (0.002)	eta 0:17:01	loss_t 0.1591 (0.3072)	loss_x 2.9317 (3.6167)	acc 50.7812 (40.7578)	lr 0.000300
epoch: [3/60][50/101]	time 0.172 (0.176)	data 0.000 (0.004)	eta 0:17:03	loss_t 0.1189 (0.2293)	loss_x 2.6051 (2.5785)	acc 67.1875 (64.3906)	lr 0.000300
epoch: [3/60][100/101]	time 0.173 (0.175)	data 0.000 (0.002)	eta 0:16:46	loss_t 0.5287 (0.2139)	loss_x 2.4147 (2.4593)	acc 64.0625 (67.4609)	lr 0.000300
epoch: [4/60][50/101]	time 0.172 (0.177)	data 0.000 (0.004)	eta 0:16:4

KeyboardInterrupt: 

In [8]:
wandb.finish()